In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
crypto_scaled = StandardScaler().fit_transform(df_market_data)
crypto_scaled

In [ ]:
# Create a DataFrame with the scaled data
crypto_transformed = pd.DataFrame(crypto_scaled, columns=['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d', 'price_change_percentage_200d',	'price_change_percentage_1y'])
crypto_transformed
# Copy the crypto names from the original data
coins = list(df_market_data.index) #this returns a list of all the coin names from the original data
coins
# Set the coinid column as index
crypto_transformed['coin_id'] = coins  #create a new column in the transformed df containing the list of coins
crypto_transformed = crypto_transformed.set_index('coin_id') #set the df index to the new coin_id column
# Display sample data
crypto_transformed

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

In [ ]:
# Create an empty list to store the inertia values
inertia1 = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(crypto_transformed)
    inertia1.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data1 = {"k": k, "inertia": inertia1}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df1 = pd.DataFrame(elbow_data1)
elbow_df1

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df1.hvplot.line(
    x="k",
    y="inertia",
    title= "Elbow Curve",
    xticks=k
)

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(crypto_transformed)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_4 = model.predict(crypto_transformed)

# Print the resulting array of cluster values.
k_4

In [ ]:
# Create a copy of the DataFrame
clusters_predicted = crypto_transformed.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
clusters_predicted['predicted_cluster'] = k_4

# Display sample data
clusters_predicted.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
clusters_predicted.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='predicted_cluster',
    hover_cols = 'coin_id'
)

In [ ]:
# Composite plot to contrast the Elbow curves
elbow_df1.hvplot.line(x="k", y="inertia", title= "Elbow Curve 1", xticks=k) + elbow_df2.hvplot.line(x="k", y="inertia", 

In [ ]:
# Composite plot to contrast the clusters
clusters_predicted.hvplot.scatter( x='price_change_percentage_24h', y='price_change_percentage_7d', by='predicted_cluster'